In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/01102018_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 99.9 ms


In [3]:
#adding this in fow now, will eventually want to switch this to the 03-clinical_variables_final
import psycopg2
import getpass
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.55 s


#### Note: Nelson has said we do not need to incorporate Echo data, so this has been removed 10/22/18

In [4]:
#reading in all of my data that is not limited to 72 hour time window between t_0 and t+72
date= '27082018'  #date of the files i will be importing

ventcategory_df=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_ventcategory_df.csv' %(date)), index_col=0)

vaso_dose_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vaso_dose_72.csv' %(date)), index_col=0)


# echodata_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date)), index_col=0)

labs_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_labs_all_nosummary_72.csv' %(date)), index_col=0)


weightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_weightfirstday_df.csv' %(date)), index_col=0)
heightfirstday_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_heightfirstday_df.csv' %(date)), index_col=0)

vitals_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_vitals_all_nosummary_72.csv' %(date)), index_col=0)
uti_all_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_uti_all_72.csv' %(date)), index_col=0)
bg_all_nosummary_72=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_bg_all_nosummary_72.csv' %(date)), index_col=0)

rrt_merged_allpt_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_rrt_merged_allpt_df.csv' %(date)), index_col=0)
gcs72_df=pd.read_csv(Path(
    wd+'/data/raw/csv/72_hr_window/%s_gcs72_df.csv' %(date)), index_col=0)

sofa_df_72=pd.read_csv(Path(
    wd+'/data/raw/csv/%s_sofa_df_72.csv' %(date)), index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 14.1 s


In [5]:
def df_subset(df):
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(patients),:]
    return(df)

time: 5 ms


In [6]:
def yn_convert(df, label_fill, pt= final_pt_df2, value_fill=0, delta_fill=pd.to_timedelta('0 days'), uom_fill='y/n'):

    yn_df = pd.merge(pt[['icustay_id', 't_0']],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left')

    yn_df['value']= yn_df['value'].fillna(value_fill)
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta'])
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)

time: 15.2 ms


In [7]:
# date that i ran the below code on the files.

date= '14112018'

time: 713 µs


## vaso dose

In [8]:
#vaso_dose_72.head()
vaso_dose_72= vaso_dose_72.rename(
    columns={'starttime':'charttime','label':'vaso_type'}
        )#.drop(labels='endtime', axis=1) 
#renaming starttime to charttime and dropping endtime

time: 21.4 ms


In [9]:
len(vaso_dose_72)

53906

time: 2.63 ms


In [10]:
vaso_dose_72.head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
92,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
93,200952.0,2139-09-25 12:00:00,2139-09-25 21:59:00,0.080068,1.962759,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 12:00:00.000000000
94,200952.0,2139-09-25 21:59:00,2139-09-26 07:38:00,0.060601,2.000000,mg,mcg/kg/min,epinephrine,2139-09-23,2 days 21:59:00.000000000
106,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000


time: 13.7 ms


In [11]:
#removing units/hour because that is a different use of vasopressin
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'units/hour',:]
vaso_dose_72 = vaso_dose_72.loc[vaso_dose_72.loc[:,'rate_uom']!= 'Uhr',:]

time: 23 ms


In [12]:
len(vaso_dose_72)

53092

time: 2.84 ms


In [13]:
vaso_dose_72['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 4.82 ms


#### removing outliers/extreme values

In [14]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:

test_group=(vaso_dose_72[['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
     .groupby(['vaso_type','rate_uom'])
         )

vaso_dose_72['std']=test_group.transform(lambda x : x.std())
vaso_dose_72['mean']=test_group.transform(lambda x : x.mean())

normal_high_value= pd.DataFrame({
    'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
    'high_value': [40, 20, 0.5, 1, 0.1, 2]
}) #found from literature, see notes

vaso_dose_72 = pd.merge(vaso_dose_72, normal_high_value, left_on='vaso_type', right_on='vaso_type')

vaso_dose_72_rmout =(vaso_dose_72[
    ~((vaso_dose_72['vaso_rate'] > vaso_dose_72['high_value']) & ((vaso_dose_72['vaso_rate']-vaso_dose_72['mean'])>= (3*vaso_dose_72['std'])))
    ])
#ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean

time: 155 ms


In [15]:
len(vaso_dose_72) #52976

53092

time: 2.15 ms


In [16]:
vaso_dose_72['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 4.58 ms


In [17]:
print(len(vaso_dose_72_rmout), len(vaso_dose_72)) #works, only excluded ~300/52662 for outliers. 


52774 53092
time: 760 µs


In [18]:
vaso_dose_72_rmout['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin'], dtype=object)

time: 4.37 ms


In [19]:
#standardizing names, dropping unneeded columns for analysis
vaso_dose_72_rmout= vaso_dose_72_rmout.drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
vaso_dose_72_rmout= vaso_dose_72_rmout.rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})

time: 44.3 ms


In [20]:
vaso_dose_72_rmout['label'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'vasopressin',
       'dopamine', 'dobutamine'], dtype=object)

time: 4.28 ms


In [21]:
epinephrine_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='epinephrine']
norepinephrine_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='norepinephrine']
phenylephrine_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='phenylephrine']
vasopressin_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='vasopressin']
dopamine_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='dopamine']
dobutamine_df= vaso_dose_72_rmout[vaso_dose_72_rmout['label']=='dobutamine']


time: 34.1 ms


In [22]:
epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')


time: 777 ms


In [23]:
norepinephrine_df.head()

,icustay_id,t_0,value,label,uom,delta
0,267059.0,2182-11-12,0.0,norepinephrine,y/n,0 days
1,278729.0,2159-11-08,0.0,norepinephrine,y/n,0 days
2,219670.0,2190-08-01,0.0,norepinephrine,y/n,0 days
3,295185.0,2154-11-11,0.0,norepinephrine,y/n,0 days
4,211472.0,2193-12-11,0.0,norepinephrine,y/n,0 days


time: 9.08 ms


In [24]:
#yn_convert(df, label_fill, pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n')

time: 553 µs


In [25]:
def yn_convert(df, label_fill, pt= final_pt_df2, value_fill=0, delta_fill=pd.to_timedelta('0 days'), uom_fill='y/n'):

    yn_df = pd.merge(pt[['icustay_id', 't_0']],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left')

    yn_df['value']= yn_df['value'].fillna(value_fill)
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta'])
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)

time: 13.6 ms


In [26]:
#date= '22102018'
# pd.DataFrame(vaso_dose_72_rmout).to_csv(Path(
#     wd+'/data/processed/%s_vaso_prepped.csv' %(date)))

pd.DataFrame(epinephrine_df).to_csv(Path(
    wd+'/data/processed/%s_epinephrine_prepped.csv' %(date)))

pd.DataFrame(norepinephrine_df).to_csv(Path(
    wd+'/data/processed/%s_norepinephrine_prepped.csv' %(date)))

pd.DataFrame(phenylephrine_df).to_csv(Path(
    wd+'/data/processed/%s_phenylephrine_prepped.csv' %(date)))

pd.DataFrame(vasopressin_df).to_csv(Path(
    wd+'/data/processed/%s_vasopressin_prepped.csv' %(date)))

pd.DataFrame(dopamine_df).to_csv(Path(
    wd+'/data/processed/%s_dopamine_prepped.csv' %(date)))

pd.DataFrame(dobutamine_df).to_csv(Path(
    wd+'/data/processed/%s_dobutamine_prepped.csv' %(date)))


time: 2.19 s


## vitals -

In [27]:
vitals_all_nosummary_72 = vitals_all_nosummary_72.rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
vitals_all_nosummary_nonull = vitals_all_nosummary_72.loc[vitals_all_nosummary_72['label'].notnull(),:]#.count() #6930 NULL values

vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull.loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='RespRate') & 
    (vitals_all_nosummary_nonull['uom']=='bpm'),'uom']='insp/min'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempC') & 
    (vitals_all_nosummary_nonull['uom']=='?C'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='Deg. F'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF') & 
    (vitals_all_nosummary_nonull['uom']=='?F'),'uom']='Deg. C'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempF'),'label']='temperature'

vitals_all_nosummary_nonull.loc[
    (vitals_all_nosummary_nonull['label']=='TempC'),'label']='temperature'

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


time: 14.2 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

In [28]:
# #writing a function to aggregate for time length

# def timewindow_agg(df, label='', roundwindow= '120min', value_col='value', transform_fxn=max):
#     df['rounded_time']=pd.to_datetime(df['charttime']).dt.round(roundwindow)
#     subset_df=(df.loc[df.loc[:,'label']==label, :])

#     #test_df4.groupby(['icustay_id','rounded_time'])['value'].apply(lambda x: max(x)).sort_values() #works!test_df4
#     #getting the rows for the max value for each rounded_time. 
#     idx= subset_df.groupby(['icustay_id','rounded_time'])['value'].transform(transform_fxn) ==subset_df['value']
#     (df.loc[df.loc[:,'label']==label, :])= subset_df[idx]
#     return(df)

time: 1.6 ms


In [29]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'icustay_id']==228393.0) &
                         (vitals_all_nosummary_nonull.loc[:,'label']=='Glucose') &
                         (vitals_all_nosummary_nonull.loc[:,'value']>99999), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='Glucose') &
                         (vitals_all_nosummary_nonull.loc[:,'value']>99998), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='Glucose') &
                         (vitals_all_nosummary_nonull.loc[:,'value']<15), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='DiasBP') &
                         (vitals_all_nosummary_nonull.loc[:,'value']<15), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='HeartRate') &
                         (vitals_all_nosummary_nonull.loc[:,'value'].between(1,29)), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='RespRate') &
                         (vitals_all_nosummary_nonull.loc[:,'value']<4), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='SysBP') &
                         (vitals_all_nosummary_nonull.loc[:,'value']<40), 'value'])=np.nan

(vitals_all_nosummary_nonull.loc[(vitals_all_nosummary_nonull.loc[:,'label']=='TempC') &
                         (vitals_all_nosummary_nonull.loc[:,'value']<28), 'value'])=np.nan

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


time: 3.69 s


In [30]:
vitals_all_nosummary_nonull = vitals_all_nosummary_nonull.loc[vitals_all_nosummary_nonull['value'].notnull(),:]#.count() #6930 NULL values


time: 1.99 s


In [31]:
pd.DataFrame(vitals_all_nosummary_nonull).to_csv(Path(
    wd+'/data/processed/%s_vitals_prepped.csv' %(date)))


time: 39.9 s


# labs -

In [32]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 5.5 ms


In [33]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_72 = pd.merge(labs_all_nosummary_72, lab_uom, left_on='label', right_on='label')


del(lab_uom)
labs_all_nosummary_72= labs_all_nosummary_72.rename(
    columns={'valuenum':'value'}) #changing valuenum to value


time: 23.4 s


In [34]:
labs_all_nosummary_72.groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL       10989
ANION GAP    mEq/L      63336
BANDS        %           3570
BICARBONATE  mEq/L      64298
BILIRUBIN    mg/dL      18218
BUN          mg/dL      64765
CHLORIDE     mEq/L      70621
CREATININE   mg/dL      65037
GLUCOSE      mg/dL      83862
HEMATOCRIT   %          80518
HEMOGLOBIN   g/dL       66677
LACTATE      mmol/L     34942
PLATELET     K/uL       59711
POTASSIUM    mEq/L      88479
PT           SECONDS    40594
PTT          sec        42759
SODIUM       mEq/L      73756
WBC          K/uL       58006
Name: uom, dtype: int64

time: 257 ms


In [35]:
#outliers

time: 639 µs


In [36]:
#summary value removal
(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==261887) & 
    (labs_all_nosummary_72.loc[:,'label']=='CHLORIDE')
    & (labs_all_nosummary_72.loc[:,'value']==3.4),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==236290) & 
    (labs_all_nosummary_72.loc[:,'label']=='CHLORIDE')
    & (labs_all_nosummary_72.loc[:,'value']==11.0),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==292769) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==28.1),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==298457) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==48.8),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==234174) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==48.7),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==290264) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==42.0),'value'])=np.nan

(labs_all_nosummary_72.loc[
    (labs_all_nosummary_72.loc[:,'icustay_id']==290264) & 
    (labs_all_nosummary_72.loc[:,'label']=='INR')
    & (labs_all_nosummary_72.loc[:,'value']==22.8),'value'])=np.nan

time: 824 ms


In [37]:
labs_all_nosummary_72= labs_all_nosummary_72.loc[labs_all_nosummary_72.loc[:,'value'].notnull(),:]


time: 138 ms


In [38]:
labs_all_nosummary_72

,subject_id,hadm_id,icustay_id,charttime,label,value,t_0,delta,uom
0,9,150750,220597,2149-11-14 07:47:00,SODIUM,137.0,2149-11-13,1 days 07:47:00.000000000,mEq/L
1,9,150750,220597,2149-11-13 03:20:00,SODIUM,130.0,2149-11-13,0 days 03:20:00.000000000,mEq/L
2,9,150750,220597,2149-11-13 16:04:00,SODIUM,129.0,2149-11-13,0 days 16:04:00.000000000,mEq/L
3,9,150750,220597,2149-11-13 19:30:00,SODIUM,131.0,2149-11-13,0 days 19:30:00.000000000,mEq/L
4,9,150750,220597,2149-11-14 01:48:00,SODIUM,133.0,2149-11-13,1 days 01:48:00.000000000,mEq/L
5,9,150750,220597,2149-11-14 06:00:00,SODIUM,137.0,2149-11-13,1 days 06:00:00.000000000,mEq/L
6,19,109235,273430,2108-08-06 01:08:00,SODIUM,138.0,2108-08-06,0 days 01:08:00.000000000,mEq/L
7,19,109235,273430,2108-08-07 05:07:00,SODIUM,139.0,2108-08-06,1 days 05:07:00.000000000,mEq/L
8,19,109235,273430,2108-08-08 05:20:00,SODIUM,141.0,2108-08-06,2 days 05:20:00.000000000,mEq/L
9,21,111970,216859,2135-02-05 04:48:00,SODIUM,134.0,2135-02-05,0 days 04:48:00.000000000,mEq/L


time: 78.8 ms


In [39]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
labs_all_nosummary_72= labs_all_nosummary_72.loc[~labs_all_nosummary_72.loc[:,'label'].isin(unwanted_values),:]

time: 112 ms


In [40]:
pd.DataFrame(labs_all_nosummary_72).to_csv(Path(
    wd+'/data/processed/%s_labs_prepped.csv' %(date)))

time: 6.78 s


## vent category -

In [41]:
ventcategory_df['uom']='mech/O2/none category'
#ventcategory_df['dtype']= 'factor'
ventcategory_df.head()

,icustay_id,t_0,vent_day1,vent_day2,vent_day3,uom
0,267059.0,2182-11-12,Mech,None,None,mech/O2/none category
1,278729.0,2159-11-08,Oxygen,Mech,None,mech/O2/none category
2,219670.0,2190-08-01,Oxygen,None,None,mech/O2/none category
3,295185.0,2154-11-11,Oxygen,None,None,mech/O2/none category
4,211472.0,2193-12-11,None,None,None,mech/O2/none category


time: 16.8 ms


In [42]:
ventcategory_df

,icustay_id,t_0,vent_day1,vent_day2,vent_day3,uom
0,267059.0,2182-11-12,Mech,None,None,mech/O2/none category
1,278729.0,2159-11-08,Oxygen,Mech,None,mech/O2/none category
2,219670.0,2190-08-01,Oxygen,None,None,mech/O2/none category
3,295185.0,2154-11-11,Oxygen,None,None,mech/O2/none category
4,211472.0,2193-12-11,None,None,None,mech/O2/none category
5,259012.0,2120-05-24,None,None,None,mech/O2/none category
6,231735.0,2186-04-13,Oxygen,Oxygen,None,mech/O2/none category
7,252135.0,2166-04-13,Oxygen,Oxygen,Oxygen,mech/O2/none category
8,293255.0,2167-10-21,Oxygen,None,None,mech/O2/none category
9,261279.0,2167-10-26,None,None,None,mech/O2/none category


time: 43 ms


In [43]:
#melting value columns into long format
ventcategory_df= ventcategory_df.melt(
    id_vars=['icustay_id','uom','t_0'],
    var_name='label',
    value_name='value')

#adding a delta column
ventcategory_df['delta']= np.nan
ventcategory_df.loc[ventcategory_df['label']=='vent_day1','delta']= pd.to_timedelta(0,'days')
ventcategory_df.loc[ventcategory_df['label']=='vent_day2','delta']= pd.to_timedelta(1,'days')
ventcategory_df.loc[ventcategory_df['label']=='vent_day3','delta']= pd.to_timedelta(2,'days')

time: 87 ms


In [44]:
ventcategory_df.head()

,icustay_id,uom,t_0,label,value,delta
0,267059.0,mech/O2/none category,2182-11-12,vent_day1,Mech,0 days 00:00:00
1,278729.0,mech/O2/none category,2159-11-08,vent_day1,Oxygen,0 days 00:00:00
2,219670.0,mech/O2/none category,2190-08-01,vent_day1,Oxygen,0 days 00:00:00
3,295185.0,mech/O2/none category,2154-11-11,vent_day1,Oxygen,0 days 00:00:00
4,211472.0,mech/O2/none category,2193-12-11,vent_day1,None,0 days 00:00:00


time: 33.5 ms


In [45]:
#collapsing into 1 column for 3 days

time: 561 µs


In [117]:
def vent_day_collapser(x):
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
        
    return(x)


time: 7.88 ms


In [118]:
ventcategory_1day_df= ventcategory_df
ventcategory_1day_df['value']=ventcategory_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_test= ventcategory_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
ventcategory_1day_test= ventcategory_1day_test.loc[ventcategory_1day_test.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort
ventcategory_1day_test['label']= 'vent_recieved'

time: 16.3 s


In [134]:
pd.DataFrame(ventcategory_1day_df).to_csv(Path(
    wd+'/data/processed/%s_ventcategory_prepped.csv' %(date)))


time: 422 ms


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [50]:
weightfirstday_df['uom']='kg'

time: 2.18 ms


In [51]:
weightfirstday_df.head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom
7,200012,51.2,51.200001,51.200001,NaN,NaN,kg
8,200014,62.0,62.000000,NaN,NaN,NaN,kg
10,200019,65.0,65.000000,NaN,NaN,NaN,kg
16,200030,113.6,113.599998,113.599998,NaN,NaN,kg
18,200033,74.0,74.000000,NaN,NaN,NaN,kg


time: 16.8 ms


In [52]:
weightfirstday_df= weightfirstday_df[weightfirstday_df['weight'].notnull()]
weightfirstday_df= weightfirstday_df[['icustay_id','weight','uom']]

time: 6.16 ms


In [53]:
weightfirstday_df['label']= 'weight'
weightfirstday_df=weightfirstday_df.rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
weightfirstday_df['delta']=pd.to_timedelta(0,'days')

#adding t_0
weightfirstday_df= pd.merge(weightfirstday_df, final_pt_df2[['icustay_id','t_0']], left_on='icustay_id', right_on='icustay_id')

time: 119 ms


In [54]:
pd.DataFrame(weightfirstday_df).to_csv(Path(
    wd+'/data/processed/%s_weight_prepped.csv' %(date)))

time: 284 ms


In [55]:
heightfirstday_df['uom']='cm'

time: 2.69 ms


In [56]:
heightfirstday_df= heightfirstday_df[heightfirstday_df['height'].notnull()]
heightfirstday_df= heightfirstday_df[['icustay_id','height','uom']]

time: 4.96 ms


In [57]:
heightfirstday_df['label']= 'height'
heightfirstday_df=heightfirstday_df.rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
heightfirstday_df['delta']=pd.to_timedelta(0,'days')
#adding t_0
heightfirstday_df= pd.merge(heightfirstday_df, final_pt_df2[['icustay_id','t_0']], left_on='icustay_id', right_on='icustay_id')

time: 65.3 ms


In [58]:
# pd.DataFrame(heightfirstday_df).to_csv(Path(
#     wd+'/data/processed/%s_height_prepped.csv' %(date)))

time: 526 µs


# UTI

In [59]:
uti_all_72['value'].unique()#seems good

array(['NEG', 'MOD', 'TR', 'POS', 'SM', 'LG', 'SMALL', ' '], dtype=object)

time: 18 ms


In [60]:
uti_all_72_pos= uti_all_72.loc[(uti_all_72.loc[:,'value']!='NEG')&
               uti_all_72.loc[:,'value'].notnull(),:]
uti_all_72_pos= uti_all_72_pos.drop_duplicates(subset=['hadm_id','value','charttime'])

time: 108 ms


In [61]:
uti_all_72_pos.head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
484,109,139061.0,51486,2141-09-11 09:15:00,MOD,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,40301,2141-09-11,0 days 09:15:00.000000000
674,117,164853.0,51486,2133-11-30 17:54:00,TR,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,570,2133-11-30,0 days 17:54:00.000000000
806,117,164853.0,51487,2133-11-30 17:54:00,POS,NaN,NaN,Nitrite,Urine,Hematology,5802-4,570,2133-11-30,0 days 17:54:00.000000000
1429,191,136614.0,51486,2196-04-12 19:15:00,MOD,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,51881,2196-04-12,0 days 19:15:00.000000000
1584,202,108295.0,51486,2145-10-22 05:39:00,MOD,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,0389,2145-10-22,0 days 05:39:00.000000000


time: 41.3 ms


In [62]:
uti_all_72_pos['label'].unique()

array(['Leukocytes', 'Nitrite'], dtype=object)

time: 3.14 ms


In [63]:
uti_nit_pos= uti_all_72_pos[uti_all_72_pos['label']=="Nitrite"]
uti_leuk_pos= uti_all_72_pos[uti_all_72_pos['label']=="Leukocytes"]

time: 6.02 ms


In [155]:
###coding any positive leukocyte or nitrate between day 1,2 or 3 for all patients. if time granularity is later needed for models, need to revisit this data to get the exact delta value

def uti_categorization(final_pt_df2):
    
    nit_df_3dayannotation= final_pt_df2[['subject_id','hadm_id','t_0']]
    nit_df_3dayannotation['uti_day1']='Neg/Not_tested'
    nit_df_3dayannotation['uti_day2']='Neg/Not_tested' 
    nit_df_3dayannotation['uti_day3']='Neg/Not_tested' 
    
    leuk_df_3dayannotation= final_pt_df2[['subject_id','hadm_id','t_0']]
    leuk_df_3dayannotation['uti_day1']='Neg/Not_tested'
    leuk_df_3dayannotation['uti_day2']='Neg/Not_tested' 
    leuk_df_3dayannotation['uti_day3']='Neg/Not_tested' 

    leuk_pos_1= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('0 days','1 days'),'hadm_id'].tolist()
    leuk_pos_2= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('1 days','2 days'),'hadm_id'].tolist()
    leuk_pos_3= uti_leuk_pos.loc[uti_leuk_pos['delta'].between('2 days','3 days'),'hadm_id'].tolist()
    
    nit_pos_1= uti_nit_pos.loc[uti_nit_pos['delta'].between('0 days','1 days'),'hadm_id'].tolist()
    nit_pos_2= uti_nit_pos.loc[uti_nit_pos['delta'].between('1 days','2 days'),'hadm_id'].tolist()
    nit_pos_3= uti_nit_pos.loc[uti_nit_pos['delta'].between('2 days','3 days'),'hadm_id'].tolist()
    
#     both_pos_1= list(set(leuk_pos_1) & set(nit_pos_1))
#     both_pos_2= list(set(leuk_pos_2) & set(nit_pos_2))
#     both_pos_3= list(set(leuk_pos_3) & set(nit_pos_3))

    leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_1),'uti_day1']='pos'
    leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_2),'uti_day2']='pos'
    leuk_df_3dayannotation.loc[leuk_df_3dayannotation.loc[:,'hadm_id'].isin(leuk_pos_3),'uti_day3']='pos'
    
    nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_1),'uti_day1']='pos'
    nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_2),'uti_day2']='pos'
    nit_df_3dayannotation.loc[nit_df_3dayannotation.loc[:,'hadm_id'].isin(nit_pos_3),'uti_day3']='pos'
    
#     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_1),'uti_day1']='both_pos'
#     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_2),'uti_day2']='both_pos'
#     uti_df_3dayannotation.loc[uti_df_3dayannotation.loc[:,'hadm_id'].isin(both_pos_3),'uti_day3']='both_pos'
    
    return(leuk_df_3dayannotation, nit_df_3dayannotation)

time: 53 ms


In [156]:
leuk_categorization, nit_categorization= uti_categorization(final_pt_df2)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

time: 577 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [143]:
leuk_categorization

,subject_id,hadm_id,t_0,uti_day1,uti_day2,uti_day3
0,24915,194424,2182-11-12,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
1,24942,178112,2159-11-08,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
2,24968,120350,2190-08-01,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
3,24776,139951,2154-11-11,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
4,24714,163632,2193-12-11,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
5,24824,194281,2120-05-24,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
7,24837,100949,2166-04-13,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
8,24785,174027,2167-10-21,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
9,24785,124880,2167-10-26,Neg/Not_tested,Neg/Not_tested,Neg/Not_tested
10,24675,147305,2132-07-28,leuk_Pos,Neg/Not_tested,Neg/Not_tested


time: 20.5 ms


In [157]:
def uti_melt(uti_categorization):
    #melting value columns into long format
    uti_categorization= uti_categorization.melt(
        id_vars=['subject_id','hadm_id','t_0'],
        var_name='label',
        value_name='value')

    #adding a delta column
    uti_categorization['delta']= np.nan
    uti_categorization.loc[uti_categorization['label']=='uti_day1','delta']= pd.to_timedelta(0,'days')
    uti_categorization.loc[uti_categorization['label']=='uti_day2','delta']= pd.to_timedelta(1,'days')
    uti_categorization.loc[uti_categorization['label']=='uti_day3','delta']= pd.to_timedelta(2,'days')
    
    #adding icustay_id
    uti_categorization= pd.merge(uti_categorization,final_pt_df2[['icustay_id','hadm_id']], left_on='hadm_id', right_on='hadm_id')
    return(uti_categorization)

time: 9.75 ms


In [158]:
leuk_melt= uti_melt(leuk_categorization)
nit_melt= uti_melt(nit_categorization)

leuk_melt['uom']='pos/neg category'
nit_melt['uom']='pos/neg category'

time: 121 ms


In [163]:
nit_melt['value'].value_counts()

Neg/Not_tested    43565
pos                 439
Name: value, dtype: int64

time: 7.17 ms


In [167]:
leuk_melt['label'].value_counts()

uti_day1    14668
uti_day3    14668
uti_day2    14668
Name: label, dtype: int64

time: 7.5 ms


### collapsing into 1 day

In [166]:
def uti_window_categorizer(x):
    if 'pos' in list(x.unique()):
        x= 'pos'
    elif 'Neg/Not_tested' in list(x.unique()):
        x= 'Neg/Not_tested'

    return(x)

#ventcategory_1day_df=ventcategory_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)

def uti_day_collapser(uti_melt):
    uti_1day_df= uti_melt
    uti_1day_df['value']=uti_melt.groupby('icustay_id',as_index=False)['value'].transform(uti_window_categorizer)
    uti_1day_df= uti_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
    uti_1day_df= uti_1day_df.loc[uti_1day_df.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort
    return(uti_1day_df)

time: 10.9 ms


In [171]:
leuk_melt_1day= uti_day_collapser(leuk_melt)
leuk_melt_1day['label']= 'leukocyte'

nit_melt_1day= uti_day_collapser(nit_melt)
nit_melt_1day['label']= 'leukocyte'

time: 31.3 s


In [170]:
leuk_melt_1day

,subject_id,hadm_id,t_0,label,value,delta,icustay_id,uom
4383,28448,177527,2153-12-23,leukocyte,Neg/Not_tested,0 days 00:00:00,200012.0,pos/neg category
33315,9514,127229,2105-02-17,leukocyte,Neg/Not_tested,0 days 00:00:00,200014.0,pos/neg category
42339,21789,112486,2178-07-09,leukocyte,Neg/Not_tested,0 days 00:00:00,200019.0,pos/neg category
37095,14987,165426,2150-11-14,leukocyte,Neg/Not_tested,0 days 00:00:00,200030.0,pos/neg category
13725,56369,198650,2198-08-11,leukocyte,Neg/Not_tested,0 days 00:00:00,200033.0,pos/neg category
33258,9960,159243,2181-08-27,leukocyte,Neg/Not_tested,0 days 00:00:00,200036.0,pos/neg category
1359,25225,147080,2179-03-17,leukocyte,Neg/Not_tested,0 days 00:00:00,200055.0,pos/neg category
3825,28494,199133,2198-02-10,leukocyte,Neg/Not_tested,0 days 00:00:00,200059.0,pos/neg category
18930,74282,121149,2134-01-24,leukocyte,Neg/Not_tested,0 days 00:00:00,200061.0,pos/neg category
26340,99052,129142,2141-03-12,leukocyte,Neg/Not_tested,0 days 00:00:00,200063.0,pos/neg category


time: 27.8 ms


In [70]:
# pd.DataFrame(uti_all_72_notnull).to_csv(Path(
#     wd+'/data/processed/%s_uti_all_72_notnull.csv' %(date)))

# pd.DataFrame(uti_categorization).to_csv(Path( 
#     wd+'/data/processed/%s_uti_prepped.csv' %(date)))   #retired 11/14/18 and split into leuk/nit seperate df's for 1 day. 

pd.DataFrame(nit_melt_1day).to_csv(Path(
    wd+'/data/processed/%s_uti_prepped.csv' %(date)))

pd.DataFrame(nit_melt_1day).to_csv(Path(
    wd+'/data/processed/%s_uti_prepped.csv' %(date)))

time: 576 ms


# bloodgas

In [71]:
# bg_all_nosummary_72=pd.read_csv(Path(
#     wd+'/data/raw/csv/72_hr_window/27082018_bg_all_nosummary_72.csv'), index_col=0)


time: 911 µs


In [72]:
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]
bg_all_nosummary_72 = bg_all_nosummary_72.rename(index=str, columns={'valueuom':'uom'})
bg_all_nosummary_72= bg_all_nosummary_72.loc[~(bg_all_nosummary_72.loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
bg_all_nosummary_72.head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
82,9,150750,220597,2149-11-13 03:34:00,BASEEXCESS,NaN,-1,mEq/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
83,9,150750,220597,2149-11-13 03:34:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
84,9,150750,220597,2149-11-13 03:34:00,LACTATE,1.80,1.8,mmol/L,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
85,9,150750,220597,2149-11-13 03:34:00,PCO2,34.00,34,mm Hg,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000
86,9,150750,220597,2149-11-13 03:34:00,PH,7.42,7.42,units,2205972149-11-13 03:34:00,2149-11-13,0 days 03:34:00.000000000


time: 982 ms


#### erroneous value removal

NOTE: these are hard coded to positioning. if anything in the bg_all_nosummary is changed than these positions need to be updated. 

In [73]:
##calcium
#fixing the calcium errors w/o hard coding
(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==249571) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==249571) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==219600) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(bg_all_nosummary_72.loc[
    (bg_all_nosummary_72.loc[:,'icustay_id']==219600) & 
    (bg_all_nosummary_72.loc[:,'label']=='CALCIUM')
    & (bg_all_nosummary_72.loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 576 ms


In [74]:
##min chloride
#converting it to a null value without hard coding
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==261887.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==261887.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==236290.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'icustay_id']==236290.0) &
                         (bg_all_nosummary_72.loc[:,'label']=='CHLORIDE') &
                         (bg_all_nosummary_72.loc[:,'valuenum']==11.0),'value'])=np.nan

time: 432 ms


In [75]:
#peep changes summary: 
bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='PEEP')&
                        (bg_all_nosummary_72['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 86.5 ms


In [76]:
#temp changes summary:

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==253821)&
                        (bg_all_nosummary_72['valuenum']==18.9),
                        'value']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==253821)&
                        (bg_all_nosummary_72['valuenum']==18.9),
                        'valuenum']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==251788)&
                        (bg_all_nosummary_72['valuenum']==10.0),
                        'value']= np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='TEMPERATURE')&
                        (bg_all_nosummary_72['icustay_id']==251788)&
                        (bg_all_nosummary_72['valuenum']==10.0),
                        'valuenum']= np.nan

time: 443 ms


In [77]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='FIO2') &
                        (bg_all_nosummary_72.loc[:,'value']=='0'),'value']=np.nan

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label']=='FIO2') &
                        (bg_all_nosummary_72.loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(0.0,1.0)),'valuenum']

bg_all_nosummary_72.loc[(bg_all_nosummary_72.loc[:,'label'].isin(['FIO2']))&
                        (bg_all_nosummary_72['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 525 ms


#### removing null values annotated abov

In [78]:
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72['value'].notnull(),:]

time: 97.4 ms


### splitting specimen out for vent vs non-vent bg data

In [79]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

bg_all_nosummary_72= pd.merge(bg_all_nosummary_72,specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary_72=bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 950 ms


In [80]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE']
bg_vent_labels=['PCO2','PaO2','PO2','FIO2','PEEP','O2FLOW']

time: 1.55 ms


In [81]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
bg_all_nosummary_72= bg_all_nosummary_72.loc[bg_all_nosummary_72.loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label'].isin(bg_vent_labels),:]


time: 135 ms


In [82]:
#value = valuenum for all specified bg tests. removing unneeded cols

bg_col=['subject_id','hadm_id','icustay_id','charttime','delta','t_0','label','valuenum','uom']
bg_ART_nosummary_72= bg_ART_nosummary_72[bg_col]
bg_all_nosummary_72= bg_all_nosummary_72[bg_col]
del(bg_col)

time: 32.4 ms


In [83]:
bg_ART_nosummary_72['label'].unique()

array(['PCO2', 'PaO2', 'FIO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 9.41 ms


In [84]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow')

#removing o2_flow from bg_ART
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='O2FLOW',:]

time: 124 ms


In [85]:
bg_ART_nosummary_72= bg_ART_nosummary_72.loc[bg_ART_nosummary_72.loc[:,'label']!='PEEP',:]

time: 24.1 ms


In [86]:
pd.DataFrame(bg_all_nosummary_72).to_csv(Path(
    wd+'/data/processed/%s_bg_all_prepped.csv' %(date)))

pd.DataFrame(bg_ART_nosummary_72).to_csv(Path(
    wd+'/data/processed/%s_bg_ART_prepped.csv' %(date)))

pd.DataFrame(o2_flow_df).to_csv(Path(
    wd+'/data/processed/%s_o2_flow_prepped.csv' %(date)))

time: 2.65 s


# RRT

In [87]:
rrt_merged_allpt_df_notnull = rrt_merged_allpt_df.loc[rrt_merged_allpt_df['rrt'].notnull(),:]
rrt_merged_allpt_df_notnull['uom']='category'
rrt_merged_allpt_df_notnull.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,icustay_id,t_0,first_charttime,rrt,uom
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category


time: 74.4 ms


In [88]:
rrt_merged_allpt_df_notnull['delta']=pd.to_timedelta(
    pd.to_datetime(rrt_merged_allpt_df_notnull['first_charttime'])-
    pd.to_datetime(rrt_merged_allpt_df_notnull['t_0']),
    'days')

time: 63.2 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [89]:
rrt_merged_allpt_df_notnull.head()

,icustay_id,t_0,first_charttime,rrt,uom,delta
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category,00:00:00
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category,04:31:00
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category,01:00:00
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category,09:32:00
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category,01:00:00


time: 20.7 ms


In [90]:
rrt_merged_allpt_df_notnull['label']= 'rrt'
rrt_merged_allpt_df_notnull=rrt_merged_allpt_df_notnull.rename(index=str, columns={'rrt':'value'})

time: 44.7 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### converting to yes/no

In [91]:
rrt_merged_allpt_df_notnull.head()

,icustay_id,t_0,first_charttime,value,uom,delta,label
7,252135.0,2166-04-13,2166-04-13 00:00:00,1.0,category,00:00:00,rrt
8,293255.0,2167-10-21,2167-10-21 04:31:00,1.0,category,04:31:00,rrt
9,261279.0,2167-10-26,2167-10-26 01:00:00,1.0,category,01:00:00,rrt
20,251526.0,2129-12-20,2129-12-20 09:32:00,1.0,category,09:32:00,rrt
31,255314.0,2181-10-11,2181-10-11 01:00:00,1.0,category,01:00:00,rrt


time: 31.6 ms


In [104]:
rrt_merged_allpt_df_notnull['icustay_id'].nunique()

1223

time: 3.33 ms


In [92]:
rrt_yn= yn_convert(rrt_merged_allpt_df_notnull,
                   label_fill='rrt')

time: 52.4 ms


In [114]:
rrt_yn['icustay_id'].nunique()

14668

time: 4.61 ms


In [113]:
rrt_yn.head()

,icustay_id,t_0,value,label,uom,delta
0,267059.0,2182-11-12,0.0,rrt,y/n,0 days
1,278729.0,2159-11-08,0.0,rrt,y/n,0 days
2,219670.0,2190-08-01,0.0,rrt,y/n,0 days
3,295185.0,2154-11-11,0.0,rrt,y/n,0 days
4,211472.0,2193-12-11,0.0,rrt,y/n,0 days


time: 10.4 ms


In [112]:
date= '14112018'
pd.DataFrame(rrt_yn).to_csv(Path(
    wd+'/data/processed/%s_rrt_prepped.csv' %(date)))

time: 329 ms


# GCS_72

In [94]:
gcs72_df['uom']='GCS_score'

time: 3.27 ms


In [95]:
list(gcs72_df)

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'ICU_admit',
 't_0',
 'admit_plus_day',
 'delta',
 'uom']

time: 3.52 ms


In [96]:
gcs72_df['label']= 'mingcs'
gcs72_df['uom']='gcs_score'
gcs72_df=gcs72_df.rename(index=str, columns={'mingcs':'value'})

time: 48.8 ms


In [97]:
gcs72_df=gcs72_df[['subject_id','hadm_id','icustay_id','delta','label','value','t_0','uom']]

time: 11.7 ms


In [98]:
pd.DataFrame(gcs72_df).to_csv(Path(
    wd+'/data/processed/%s_gcs_prepped.csv' %(date)))

time: 81.8 ms


# SOFA_72
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [99]:
sofa_df_72['uom']='daily_sofa_score'

#adding day delta column
sofa_df_72=sofa_df_72.sort_values(['hadm_id','day','t_0']) #good
sofa_df_72['day_rank']=sofa_df_72.groupby('icustay_id')['day'].rank()
sofa_df_72['delta']=pd.to_timedelta((sofa_df_72['day_rank']-1), 'days')

time: 228 ms


In [100]:
sofa_df_72['label']= 'daily_sofa'
sofa_df_72=sofa_df_72.rename(index=str, columns={'sofa':'value'})

time: 122 ms


In [101]:
sofa_score_72= sofa_df_72[['subject_id','hadm_id','icustay_id','delta','label','value','t_0','uom']]

time: 12.4 ms


In [102]:
sofa_score_72

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
63554,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
36471,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
36472,23018,100007,217937,1 days,daily_sofa,3,2145-03-31,daily_sofa_score
36473,23018,100007,217937,2 days,daily_sofa,1,2145-03-31,daily_sofa_score
670,533,100009,253656,0 days,daily_sofa,5,2162-05-17,daily_sofa_score
671,533,100009,253656,1 days,daily_sofa,1,2162-05-17,daily_sofa_score
672,533,100009,253656,2 days,daily_sofa,2,2162-05-17,daily_sofa_score
71986,68591,100016,217590,0 days,daily_sofa,2,2188-05-25,daily_sofa_score
71987,68591,100016,217590,1 days,daily_sofa,2,2188-05-25,daily_sofa_score
71988,68591,100016,217590,2 days,daily_sofa,0,2188-05-25,daily_sofa_score


time: 46.9 ms


In [103]:
pd.DataFrame(sofa_score_72).to_csv(Path(
    wd+'/data/processed/%s_sofa_prepped.csv' %(date)))

time: 706 ms
